In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch

In [2]:
import configparser
import os 
from pathlib import Path
import pandas as pd
import json
import re
import numpy as np
from transformers import PreTrainedTokenizerFast

base_dir = Path(os.getcwd()).resolve().parent.parent
config = configparser.ConfigParser()
print(base_dir)
config.read(os.path.join(base_dir,'config.ini'))

path_file_abstract_datasets = '/home/lamedinaa/testing_rl/data/4_random_forest_model/datasets/abstract_episodes_4_combine_khordoo_252106052025.csv'
# path_file_abstract_datasets = '/home/lamedinaa/testing_rl/data/4_random_forest_model/datasets/abstract_episodes_4_combine_kapil_361502052025.csv'
# path_file_abstract_datasets = '/home/lamedinaa/testing_rl/data/4_random_forest_model/datasets/abstract_episodes_4_nihal_351905032025.csv'
print(path_file_abstract_datasets)

path_file_abstract_states = '/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_4_combine_khordoo_252106052025.json'
# path_file_abstract_states = '/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_4_combine_kapil_361502052025.json'
# path_file_abstract_states = '/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_4_nihal_351905032025.json'
print(path_file_abstract_states)

f = open(path_file_abstract_states,'r')
dict_abstract_states = json.load(f)

/home/lamedinaa/testing_rl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/lamedinaa/testing_rl
/home/lamedinaa/testing_rl/data/4_random_forest_model/datasets/abstract_episodes_4_combine_khordoo_252106052025.csv
/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_4_combine_khordoo_252106052025.json


## Tokenize

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

## definimos el vocabularion
especial_words = ["1","0","2","3","True",'[UNK]','[BOS]']
vocabulary_map = {f'w{i+1}':abstract_class for i,abstract_class in enumerate(dict_abstract_states.keys()) }
vocabulary = list(vocabulary_map.keys()) + especial_words 
## tokenizamos
vocab =  {word:idx for idx,word in enumerate(vocabulary)}

tokenizer = Tokenizer(WordLevel(vocab,unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

print(len(vocabulary))

946


In [4]:
len(vocab.keys())

946

In [5]:
#### get and prepare data
df_abstract_states = pd.read_csv(path_file_abstract_datasets,sep=';')

def clean_text(row):
    return '[BOS] '+re.sub(r'\[\[.*?\]\]', '[UNK]', row['parse_abstract_states']) + ' True'

texts = df_abstract_states.apply(clean_text,axis=1)

###### CODIFICACIÓN DE TEXTOS 
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object = tokenizer)
fast_tokenizer.add_special_tokens({
    'unk_token': '[UNK]',
    'pad_token': '[PAD]',
    'bos_token': '[BOS]'
})
encodings = fast_tokenizer(
    list(texts),
    padding="max_length",
    truncation=True, 
    max_length=1024, 
    return_tensors='pt'
)


def scale(row,min_reward,max_reward):
    reward = 200 if row['reward_mean']>200 else row['reward_mean']
    reward = 0 if reward<0 else reward
    return (reward - min_reward)/(max_reward - min_reward)

def scale2(row): 
    if row['prob_fault']<0: 
        return 0
    if row['prob_fault'] > 1: 
        return 1        
    return row['prob_fault']

rewards = df_abstract_states.apply(lambda row: scale(row,0,200),axis=1) ## norm rewards
# prob_fault = df_abstract_states['prob_fault']
prob_fault = df_abstract_states.apply(scale2,axis=1)

######## PRUEBA
ids = [int(t) for t in list(encodings['input_ids'][0])]
print(tokenizer.decode(ids))
print(texts[0])

[BOS] w384 3 w384 3 w241 3 w397 1 w241 3 w183 3 w346 1 w183 3 w183 3 w346 1 w183 3 w203 0 w203 0 w203 0 w130 0 w130 0 w130 0 w130 0 w242 0 w242 0 w242 0 w242 0 w350 0 w133 2 w133 2 w350 0 w133 2 w133 2 w242 0 w205 2 w242 0 w205 2 w242 0 w133 2 w205 2 w242 0 w133 2 w242 0 w205 2 w205 2 w57 2 w130 0 w57 2 w57 2 w57 2 w203 0 w130 0 w57 2 w57 2 w203 0 w203 0 w57 2 w203 0 w57 2 w57 2 w203 0 w96 2 w217 0 w96 2 w96 2 w217 0 w217 0 w96 2 w346 1 w57 2 w96 2 w217 0 w151 2 w151 2 w131 2 w159 1 w74 0 w151 2 w151 2 w131 2 w31 2 w89 1 w103 0 w131 2 w131 2 w31 2 w118 2 w110 1 w31 2 w81 0 w31 2 w118 2 w118 2 w166 0 w44 2 w44 2 w166 0 w44 2 w53 2 w119 1 w102 0 w53 2 w55 0 w53 2 w16 2 w55 0 w16 2 w99 0 w16 2 w16 2 w16 2 w99 0 w34 1 w99 0 w16 2 w99 0 w147 2 w99 0 w147 2 w34 1 w147 2 w39 2 w132 0 w39 2 w39 2 w168 0 w164 2 w168 0 w164 2 w164 2 w46 0 w28 2 w37 0 w37 0 w28 2 w37 0 w52 2 w37 0 w52 2 w40 2 w141 0 w40 2 w141 0 w40 2 w4 0 w32 2 w32 2 w32 2 w4 0 w32 2 w32 2 w32 2 w85 0 w66 2 w66 2 w66 2 w66 2 w38

In [6]:
for t in list(encodings['input_ids']): 
    if t.min().item() <0 or t.max().item()>946: 
        print(f"Min: {t.min().item()}, Max: {t.max().item()}")

In [7]:
len(fast_tokenizer)

947

In [8]:
for r in list(rewards):
    if r<0 or r>1:
        print(r)

In [9]:
for p in list(prob_fault):
    if p<0 or p>1:
        print(p)
        

## CONDITIONAL DATASET

In [10]:
import torch
from torch.utils.data import Dataset

class CondDataset(Dataset):
    def __init__(self, encodings, rewards, probs):
        """
        encodings: dict con 'input_ids' y 'attention_mask' (tensors pt)
        rewards:  lista o tensor de floats normalizados en [0,1]
        probs:    lista o tensor de floats normalizados en [0,1]
        """
        self.encodings = encodings
        self.rewards   = torch.tensor(rewards, dtype=torch.float)
        self.probs     = torch.tensor(probs,   dtype=torch.float)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids":      self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels":         self.encodings["input_ids"][idx],  # causal LM
            "reward":         self.rewards[idx],
            "prob_fault":     self.probs[idx],
        }


dataset = CondDataset(encodings,rewards,prob_fault)

In [11]:
dataset.__getitem__(1)

{'input_ids': tensor([945,  33, 939,  ..., 946, 946, 946]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor([945,  33, 939,  ..., 946, 946, 946]),
 'reward': tensor(1.),
 'prob_fault': tensor(0.0880)}

In [12]:
dataset.__getitem__(1)

{'input_ids': tensor([945,  33, 939,  ..., 946, 946, 946]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor([945,  33, 939,  ..., 946, 946, 946]),
 'reward': tensor(1.),
 'prob_fault': tensor(0.0880)}

In [13]:
dataset.__getitem__(1)['input_ids']

tensor([945,  33, 939,  ..., 946, 946, 946])

In [14]:
dataset.__getitem__(1)['input_ids']

tensor([945,  33, 939,  ..., 946, 946, 946])

In [15]:
len(dataset.__getitem__(1)['input_ids']),len(dataset.__getitem__(1)['labels'])

(1024, 1024)

In [16]:
len(dataset.__getitem__(1)['input_ids']),len(dataset.__getitem__(1)['labels'])

(1024, 1024)

## MODEL

In [17]:
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Config

class CondGPT2(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
        d = config.n_embd
        self.cond_proj = nn.Sequential(
            nn.Linear(2, d),
            nn.ReLU(),
            nn.Linear(d, d)
        )

    def forward(self, input_ids, attention_mask=None,
                labels=None, reward=None, prob_fault=None, **kwargs):
                

        print('debug 1')
        cond = torch.stack([reward, prob_fault], dim=-1)     
        cond_emb = self.cond_proj(cond)                    
        print('debug 2')
        inputs_embeds = self.transformer.wte(input_ids)    
        print('debug 3')
        cond_exp = cond_emb.unsqueeze(1).expand_as(inputs_embeds)
        inputs_embeds = inputs_embeds + cond_exp
        print('debug 4')
        return super().forward(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )

    def prepare_inputs_for_generation(self,
            input_ids,
            attention_mask=None,
            **model_kwargs
        ):
        # Este método se invoca en cada paso de generate().
        # Sacamos reward y prob_fault de model_kwargs
        print('debug 5')
        return {
            "input_ids":      input_ids,
            "attention_mask": attention_mask,
            "reward":         model_kwargs["reward"],
            "prob_fault":     model_kwargs["prob_fault"],
        }


2025-05-11 02:35:44.963429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-11 02:35:45.096977: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 02:35:45.781346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-05-11 02:35:45.781408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# Training

In [18]:
# Después de model.to(device), pero antes de trainer.train():
for i in range(5):
    batch = dataset[i]
    ids = batch['input_ids']
    print(f"Batch {i}: min_id={ids.min().item()}, max_id={ids.max().item()}", 
          f"dtype={ids.dtype}")



Batch 0: min_id=0, max_id=946 dtype=torch.int64
Batch 1: min_id=0, max_id=946 dtype=torch.int64
Batch 2: min_id=0, max_id=946 dtype=torch.int64
Batch 3: min_id=0, max_id=946 dtype=torch.int64
Batch 4: min_id=0, max_id=945 dtype=torch.int64


In [19]:
print("pad_token_id =", fast_tokenizer.pad_token_id)


pad_token_id = 946


In [20]:
from transformers import Trainer, TrainingArguments, default_data_collator
from transformers import GPT2Config, GPT2LMHeadModel


from torch.utils.data import Dataset,DataLoader,TensorDataset


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
save_directory = '/home/lamedinaa/testing_rl/data/5_models_LP/transformers/gpt2_model3'

config = GPT2Config(
    vocab_size=len(fast_tokenizer),
    n_embd=512,
    n_layer=6,
    n_head=8,
    bos_token_id=fast_tokenizer.pad_token_id,
    eos_token_id=fast_tokenizer.pad_token_id,
    pad_token_id=fast_tokenizer.pad_token_id
)
model = CondGPT2(config)
model.resize_token_embeddings(len(fast_tokenizer))
model = model.to(device)


training_args = TrainingArguments(
    output_dir = save_directory,
    num_train_epochs = 10,
    per_device_train_batch_size =8, 
    save_steps=500, 
    save_total_limit=2, 
)

trainer = Trainer(
    model = model,
    args= training_args, 
    train_dataset=dataset,
    data_collator = default_data_collator
)



trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


debug 1
debug 2
debug 3
debug 4


Step,Training Loss


debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1
debug 2
debug 3
debug 4
debug 1


KeyboardInterrupt: 

## SAVE MODEL

In [3]:
import os

model_path = '/home/lamedinaa/testing_rl/data/5_models_LP/prueba/conditional_gpt_models/nihal'
os.makedirs(model_path,exist_ok=True)
model.save_pretrained(model_path)
fast_tokenizer.save_pretrained(model_path)


NameError: name 'model' is not defined

## PROMPTING

In [4]:
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Config

class CondGPT2(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
        d = config.n_embd
        self.cond_proj = nn.Sequential(
            nn.Linear(2, d),
            nn.ReLU(),
            nn.Linear(d, d)
        )

    def forward(self, input_ids, attention_mask=None,
                labels=None, reward=None, prob_fault=None, **kwargs):


        cond = torch.stack([reward, prob_fault], dim=-1)     
        cond_emb = self.cond_proj(cond)                    

        inputs_embeds = self.transformer.wte(input_ids)    

        cond_exp = cond_emb.unsqueeze(1).expand_as(inputs_embeds)
        inputs_embeds = inputs_embeds + cond_exp
        
        return super().forward(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )

    def prepare_inputs_for_generation(self,
            input_ids,
            attention_mask=None,
            **model_kwargs
        ):
        # Este método se invoca en cada paso de generate().
        # Sacamos reward y prob_fault de model_kwargs
        return {
            "input_ids":      input_ids,
            "attention_mask": attention_mask,
            "reward":         model_kwargs["reward"],
            "prob_fault":     model_kwargs["prob_fault"],
        }


/home/lamedinaa/testing_rl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 00:20:49.192298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-11 00:20:49.324557: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 00:20:49.977259: W tensorflow/compiler/xla/stream_executor/platform/

In [5]:
from transformers import Trainer, TrainingArguments, default_data_collator
from transformers import GPT2Config, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset,DataLoader,TensorDataset
from transformers import PreTrainedTokenizerFast

### nihal
# save_dir = '/home/lamedinaa/testing_rl/data/5_models_LP/models/model_condtransformer_4_nihal_351905032025'
### nihal
# save_dir = '/home/lamedinaa/testing_rl/data/5_models_LP/models/model_condtransformer_4_combine_nihal_291602052025'
### kapil
save_dir = '/home/lamedinaa/testing_rl/data/5_models_LP/models/model_condtransformer_4_combine_kapil_361502052025'
### shakti
# save_dir = '/home/lamedinaa/testing_rl/data/5_models_LP/models/model_condtransformer_4_combine_shakti_051702052025'
config = GPT2Config.from_pretrained(save_dir)
model_charged = CondGPT2.from_pretrained(save_dir, config=config)
# model_charged.to("cuda" if torch.cuda.is_available() else "cpu")
model_charged.eval()

fast_tokenizer = PreTrainedTokenizerFast.from_pretrained('/home/lamedinaa/testing_rl/data/5_models_LP/models/model_condtransformer_4_nihal_351905032025')

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

t = torch.tensor([1,2], device=device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [78]:
model_charged.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_charged.to(device)

# 1) Prompt y tokenización inicial
prompt = "[BOS]"
inputs = fast_tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model_charged.device)
attention_mask = inputs["attention_mask"].to(model_charged.device)

# 2) Tensor de condición (batch_size=1)
reward_tensor     = torch.tensor([190/200], device=model_charged.device)  # normalizado
prob_fault_tensor = torch.tensor([0], device=model_charged.device)

# reward_tensor = torch.tensor([190/200]) 
# prob_fault_tensor = torch.tensor([0.8])

# 3) Generación
outputs = model_charged.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    reward=reward_tensor,        # aquí pasas tu condición
    prob_fault=prob_fault_tensor,
    max_length=300,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    num_return_sequences=20,
)

# 4) Decodificar
for i, out in enumerate(outputs):
    text = fast_tokenizer.decode(out, skip_special_tokens=True)
    print(f"Generación {i+1}:", text)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [53]:
model_charged.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_charged.to(device)

# 1) Prompt y tokenización inicial
prompt = "[BOS]"
inputs = fast_tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model_charged.device)
attention_mask = inputs["attention_mask"].to(model_charged.device)

# 2) Tensor de condición (batch_size=1)
reward_tensor     = torch.tensor([90/200], device=model_charged.device)  # normalizado
prob_fault_tensor = torch.tensor([1], device=model_charged.device)

# reward_tensor = torch.tensor([190/200]) 
# prob_fault_tensor = torch.tensor([0.8])

# 3) Generación
outputs = model_charged.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    reward=reward_tensor,        # aquí pasas tu condición
    prob_fault=prob_fault_tensor,
    max_length=300,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    num_return_sequences=20,
)

# 4) Decodificar
for i, out in enumerate(outputs):
    text = fast_tokenizer.decode(out, skip_special_tokens=True)
    print(f"Generación {i+1}:", text)

Generación 1: w20 1 w20 0 w20 1 w28 0 w28 1 w28 0 w12 0 w12 1 w12 0 w17 1 w5 0 w12 1 w18 0 w18 1 w18 0 w18 1 w17 0 w17 0 w17 1 w17 0 w1 1 w17 0 w1 1 w1 0 w1 1 w1 0 w31 1 w31 0 w31 0 w31 1 w4 0 w4 1 w4 0 w4 1 w4 0 w2 1 w2 0 w2 0 w2 1 w14 0 w14 1 w14 0 w14 1 w8 0 w8 1 w8 0 w8 1 w22 0 w22 0 w22 1 w22 0 w22 1 w26 0 w26 0 w26 0 w26 1 w15 0 w15 1 w15 0 w15 0 w25 1 w25 0 w25 1 w25 0 w19 1 w19 0 w19 0 w19 1 w5 0 w5 1 w5 1 w5 0 w5 0 w11 1 w11 0 w11 0 w11 1 w11 1 w13 0 w13 0 w13 1 w10 0 w10 1 w10 0 w10 1 w10 0 w9 0 w9 1 w9 0 w9 1 w27 0 w27 1 w27 0 w27 1 w16 0 w16 0 w16 1 w16 0 w3 1 w3 0 w3 1 w3 0 w6 1 w6 0 w6 1 w6 0 w7 0 1 w7 0 True
Generación 2: w32 1 w20 0 w20 1 w20 0 w28 1 w28 0 w28 1 w28 0 w12 0 w12 1 w12 0 w12 1 w18 0 w18 1 w18 0 w18 1 w18 0 w17 0 w17 1 w17 0 w17 1 w17 0 w1 1 w1 0 w1 1 w1 0 w1 1 w31 1 w31 0 w31 0 w31 1 w4 1 w4 0 w4 1 w4 0 w4 1 w2 0 w2 1 w2 0 w2 1 w2 0 w14 0 w14 1 w14 0 w14 1 w8 0 w8 1 w8 0 w8 1 w22 0 w22 1 w22 0 w22 0 w22 1 w26 0 w26 1 w26 0 w26 1 w15 0 w15 0 w15 1 w15 0 w2

In [5]:
model_charged.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_charged.to(device)

# 1) Prompt y tokenización inicial
prompt = "[BOS]"
inputs = fast_tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model_charged.device)
attention_mask = inputs["attention_mask"].to(model_charged.device)

# 2) Tensor de condición (batch_size=1)
reward_tensor     = torch.tensor([180/200], device=model_charged.device)  # normalizado
prob_fault_tensor = torch.tensor([1], device=model_charged.device)

# reward_tensor = torch.tensor([190/200]) 
# prob_fault_tensor = torch.tensor([0.8])

# 3) Generación
outputs = model_charged.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    reward=reward_tensor,        # aquí pasas tu condición
    prob_fault=prob_fault_tensor,
    max_length=300,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    num_return_sequences=20,
)

# 4) Decodificar
for i, out in enumerate(outputs):
    text = fast_tokenizer.decode(out, skip_special_tokens=True)
    print(f"Generación {i+1}:", text)

Generación 1: w37 1 w39 1 w11 1 w36 0 w37 1 w36 1 w36 0 w36 0 w37 1 w37 0 w36 1 w35 0 w35 0 w36 1 w35 0 w40 1 w35 0 w35 1 w35 0 w39 1 w33 0 w35 0 w33 1 w33 1 w33 0 w33 1 w33 0 w34 1 w34 0 w34 1 w29 0 w34 1 w34 0 w29 1 w29 0 w29 1 w29 1 w29 0 w29 0 w21 1 w29 1 w21 1 w21 0 w21 1 w21 0 w21 1 w21 1 w24 1 w24 1 w24 0 w24 0 w24 0 w24 1 w30 1 w24 0 w30 0 w30 1 w30 1 w30 0 w30 1 True 0 w23 0 w23 0 w23 1 w23 0 w32 1 w32 0 w32 1 w32 0 w20 0 w20 1 w20 0 w20 1 w20 0 w28 1 w28 0 w28 0 w28 1 w12 1 w28 1 w12 0 w12 0 w18 0 w18 0 w18 1 w18 1 w18 0 w17 1 w17 0 w17 1 w17 0 w1 1 w1 0 w1 1 w1 1 w31 0 w31 1 w4 0 w31 0 w4 1 w4 1 w4 0 w4 0 w4 0 w4 1 w2 0 w2 1 w2 0 w2 1 w14 1 w14 0 w14 1 w14 0 w14 1 w14 0 w8 1 w8 0 w8 0 w8 0 w8 1 w8 0 w22 1 w22 0 w26 1 w26 0 w26 1 w15 0 w15 1 w15 0 w15 0 w15 1 w15 0 w25 1 w25 0 w25 1 w25 0 w19 1 w19 0 w19 1 w5 0 w5 0 w5 0 w5 1 w5 0 w11 1 w5 0 w11 0 w11 1 w11 0 w13
Generación 2: w36 1 w35 1 w37 1 w40 1 w39 0 w37 1 w37 0 w39 1 w37 0 w37 0 w36 1 w37 0 w38 1 w40 0 w36 1 w36 0 w36 